In [1]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!ls /content/drive/My\ Drive/Computadora\ Virtual/Data\ Analytics/Investigacion/Parkinson/Data/

ls: cannot access '/content/drive/My Drive/Computadora Virtual/Data Analytics/Investigacion/Parkinson/Data/': No such file or directory


# Import libraries

In [3]:
!pip install h2o

     |████████████████████████████████| 164.8 MB 3.9 kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.5-py2.py3-none-any.whl size=164886106 sha256=0012d2a76f81837aed8451d80fd27f5dcd7fccf2514bc0d98c89d2735952f9af
  Stored in directory: /root/.cache/pip/wheels/2f/f4/f6/7115a720596f0b6c377b3d82c28242585c7bb7ab27d430f97c
Successfully built h2o


In [4]:
# Reinicia entorno de Ejecución
for name in dir():
       if not name.startswith('_'):
           del globals()[name]
           
import pandas as pd
import numpy as np

from random import seed
from random import randint
import random

import time

import h2o
h2o.init()
from h2o.automl import H2OAutoML

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5_t23f82
  JVM stdout: /tmp/tmp5_t23f82/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5_t23f82/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,"14 days, 7 hours and 3 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_4pcroi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


# Training and Testing Function

In [5]:
############################################
########   Juan A. Nolazco Flores   ########
############################################
def agument_Data(data, Eps, proportion):
  #print('proportion=', proportion)
  N = int(proportion*data.shape[0])
  df_to_be_agumented = data
  rn = rng.normal(0, Eps, size=(1, data.shape[1]))[0]
  #print('rn=',rn,'len(rn)',len(rn))

  for i in range(N):                              # create new N data. Where N are the required proportion.
    rd_row=random.randint(0,len(data)-1)             # randomly select one row to include noise.
    #print('i',i,'rd_row',rd_row)
    #print('data.values[rd_row,0:data.shape[1]]=',data.values[rd_row,0:data.shape[1]])
    b = data.values[rd_row,0:data.shape[1]] + rn  # add Gaussian Noise to the selected observation.
    #print('1b=',b)
    b = pd.DataFrame(b)                      # return format to Panda Dataframe
    b=b.T
    #print('2b=',b)

    b.columns = data.columns                         # re-name the new dataframe with the original names of the columns
    #print('3b=',b)
    b.values[0,b.shape[1]-1] = int(data.values[rd_row,b.shape[1]-1]) # convert to integer the last value (this is the value used to classify)
    #print('4b=',b)
    df_to_be_agumented = pd.concat([df_to_be_agumented,b])      # concat the augmente observation at the end of the array.
    df_to_be_agumented.reset_index(inplace=True, drop=True)     # reset the index
  return(df_to_be_agumented)

In [6]:
####################################################################################
########################   Juan A. Nolazco Flores   ################################
####################################################################################
def make_dataframes_classes_same_length(training_data, Eps, prop):  # data must be the training data
  e0 = training_data[(training_data['Stress_Expression']==0)]   # data no presenting the emotion.
  e1 = training_data[(training_data['Stress_Expression']==1)]   # data presenting the emotion at moderate level.
  e2 = training_data[(training_data['Stress_Expression']==2)]   # NEW: data presenting the emotion at high level.
  l_e0 = len(e0)
  l_e1 = len(e1)
  l_e2 = len(e2)

  print('e0',len(e0))
  print('e1',len(e1))
  print('e2',len(e2))
  if (l_e0>l_e1)and(l_e0>l_e2):
      prop = l_e0/l_e1-1
      e1=agument_Data(e1, Eps, prop)
      prop = l_e0/l_e2-1
      e2=agument_Data(e2, Eps, prop)
  elif (l_e1>l_e0)and(l_e1>l_e2):
      prop = l_e1/l_e0-1
      e0=agument_Data(e0, Eps, prop)
      prop = l_e1/l_e2-1
      e2=agument_Data(e2, Eps, prop)
  else:
      prop = l_e2/l_e0-1
      e0=agument_Data(e0, Eps, prop)
      prop = l_e2/l_e1-1
      e2=agument_Data(e2, Eps, prop)  
  
  df_to_be_agumented = pd.concat([e0,e1,e2])                   # concat the augmente observation at the end of the array.
  df_to_be_agumented.reset_index(inplace=True, drop=True)   # reset the index
  return(df_to_be_agumented)

In [7]:
####################################################################################
########################   Juan A. Nolazco Flores   ################################
####################################################################################
def train_test_LOOnd(data, test_ctl_pd, n, num_testing_samples):
  print('test_ctl_pd.shape[0]=', test_ctl_pd.shape[0])
  #part_length = int(test_ctl_pd.shape[0]/num_testing_samples) 
  #print('part_length', part_length)

  test  = test_ctl_pd.iloc[:num_testing_samples]                # get the top "part_length" users for testing.
  # Constructing the training data:
  train1 = data.iloc[                         :num_testing_samples*n]  # get the top "part_length" users for training.
  train2 = data.iloc[num_testing_samples*(n+1):                     ]  # get the bottom "part_length" users for traing.
  train = pd.concat([train1, train2],axis=0)                           # concatenatig the two parts to form the training data.
  
  test_ctl_pd = test_ctl_pd.iloc[num_testing_samples:]        # delete the top "part_length" from the train_ctl_pd for the next iteration.
  
  #print('test.shape[0] =', test.shape[0])
  #print('train.shape[0]=', train.shape[0])


  return(train, test, test_ctl_pd)    

# **Main**

In [ ]:
####################################################################################
########################   Juan A. Nolazco Flores   ################################
####################################################################################
start_time=time.time()

path = "/content/drive/My Drive/JnOTE/graficos/"
name='oth0.18_ith0.7_PCA_Em0_KF_SF_CDF_SDF_Split3_nf100_bwf1_fi0.5_ov0_bdfb15_Stress_Expression'
file= path + "Emo3class/3Segments/" + name + ".csv"
container = []
#results_file = open(path + 'Results/' + 'result_' + file +"_15per_prop_01"+ '.txt', 'a')

#file= path + "Data/TotalMatrix_S2320_C720_norm.csv"
#PaHaw_data = pd.read_csv(path+file+'.csv')
data = pd.read_csv(file)
data = data.dropna(axis=1)                                               # Dropping the columns having NaN/NaT values
data=data.iloc[np.random.permutation(data.index)].reset_index(drop=True) # Randomly Shuffle DataFrame Rows to avoid one of the folds only contains one category.

fbl = 0

#data=data._get_numeric_data()  # keep only columns which all values are numeric.
h2o_df = h2o.H2OFrame(data)     # convert panda dataframe to use the h2o machine learning plataform.
h2o.no_progress()
##h2o.show_progress()

nf = data.shape[1]-1            # número de features (componentes)
nc = nf                         # número de componetes (features) a utilizar
col = list(range(nc))

per_testing_set = 15            # Percentage of the testing rate.
prop_testing_set = per_testing_set/100
num_testing_samples = int(data.shape[0]*(prop_testing_set))   # Number of obervation used for testing.
print('num_testing_samples')

k=1
acc_max=0
for w in range(1,200):
   for j in range(w+1,w+10):
     print('&&&&&&&& w=',w,'j=',j, '&&&&&&&&&&&&&&&&&&&&&&&&&&&& Execution time: ', time.time()-start_time)
     for prop in [0.1]:
       if fbl>=1: print('prop=',prop)
       for Eps in [0.1, 0.2, 0.3, 0.4, 0.5]: # std deviation of the Gaussian Noise.
        if (fbl == 1): print('Eps=',Eps)
        random.seed(w)
        rng = np.random.RandomState(j)
        acc_acu=0
        test_ctl_pd = data # initialize test_ctl_pd with all the users... in each iteration test_ctl_pd file will be deletting the testing files.
        i = 0
        N = int(data.shape[0]/num_testing_samples)  # The number of iterations for co-evaluation depend on the num_obs (number of observations used for testing)
        print('N=',N,'data.shape[0]',data.shape[0],'num_testing_samples=',num_testing_samples)
        for n in range(0,N):
          if fbl>=1: print(i,'.',end='')
          i+=1
          start_time2=time.time()
          train, test, test_ctl_pd = train_test_LOOnd(data, test_ctl_pd, n, num_testing_samples)

          if fbl>=1: print('train.shape[0]',train.shape[0])

          train = make_dataframes_classes_same_length(train, Eps, prop)
          e0 = train[(train['Stress_Expression']==0)]   # data no presenting the emotion.
          e1 = train[(train['Stress_Expression']==1)]   # data presenting the emotion.
          e2 = train[(train['Stress_Expression']==2)]   # data presenting the emotion.
          l_e0 = len(e0)
          l_e1 = len(e1)
          l_e2 = len(e2)
          if fbl>=1: print('train.shape[0]',train.shape[0], 'no=',l_e0,'moderate=',l_e1, 'high=',l_e2,)
          train = agument_Data(train, Eps, prop)
          e0 = train[(train['Stress_Expression']==0)]   # data no presenting the emotion.
          e1 = train[(train['Stress_Expression']==1)]   # data presenting the emotion.
          e2 = train[(train['Stress_Expression']==2)]   # data presenting the emotion.
          l_e0 = len(e0)
          l_e1 = len(e1)
          l_e2 = len(e2)
          if fbl>=1: print('train.shape[0]',train.shape[0], 'no=',l_e0,'moderate=',l_e1, 'high=',l_e2,)

          test.loc[(test.Stress_Expression)   == 0, 'Stress_Expression'] = 'no'
          test.loc[(test.Stress_Expression)   == 1, 'Stress_Expression'] = 'moderate' # this assignment was needed, when the dataframe is convert to h2o dataframes.
          test.loc[(test.Stress_Expression)   == 2, 'Stress_Expression'] = 'high'     # this assignment was needed, when the dataframe is convert to h2o dataframes.
          train.loc[(train.Stress_Expression) == 0, 'Stress_Expression'] = 'no'
          train.loc[(train.Stress_Expression) == 1, 'Stress_Expression'] = 'moderate' # this assignment was needed, when the dataframe is convert to h2o dataframes.
          train.loc[(train.Stress_Expression) == 2, 'Stress_Expression'] = 'high'     # this assignment was needed, when the dataframe is convert to h2o dataframes.
          train=train.reset_index(drop=True)

          train_shuffled=train.iloc[np.random.permutation(train.index)].reset_index(drop=True) # Randomly Shuffle DataFrame Rows to avoid one of the folds only contains one category.


          test_h2o  = h2o.H2OFrame(test)                     # convert panda dataframe to h20 dataframe
          train_h2o = h2o.H2OFrame(train_shuffled)           # convert panda dataframe to h20 dataframe

          print('time.time()-start_time=', time.time()-start_time2)

          # Identify predictors and response
          x = train_h2o.columns
          y = "Stress_Expression"
          x.remove(y)

          # model definition
          aml = H2OAutoML(max_runtime_secs=200,
                  max_models=15,
                  exclude_algos = ['GBM'],
                  seed=1,
                  nfolds=2,
                  balance_classes=True # to minize the categories balance problem . ¿balance_classes=True, class_sampling_factors=[1.0,1.0,2.5]?
                  #sort_metric='mean_per_class_error',
                  #stopping_metric="logloss", # logloss (this is the devalut for Classification), MSE, MAE; RMSLE, AUC, AUCPR, lift_top_group, misClassification, maean_per_Class_error
                  #sort_metric='auc',    # mse, MAE; RMSLE, auc, aucpr, lift_top_group, misclasification, maean_per_Class_error
                  #project_name='Completed'
                  )
          
          # model trining
          #%time aml.train(x=x, y=y, training_frame=train)
          aml.train(x=x, y=y, training_frame=train_h2o)

          lb = aml.leaderboard
          if fbl>=1: print('lb.head',lb.head(rows=lb.nrows))  # Print all rows instead of default (10 rows)

          #print('i=',i,'test=',test)
          model = h2o.get_model(lb[0,0])      # Select the leader model to test.

          cma=model.model_performance(test_h2o).confusion_matrix()
          # print('cma=',cma)
          true_positives=(cma['high'][0]+cma['moderate'][1]+cma['no'][2])
          acc = true_positives/test.shape[0]
          acc_acu += acc
          acc_m = acc_acu/i
          # print('****** w=',w,'j=',j,'n=',n,'prop=',prop,'Eps=',Eps,'***********', 'acc',acc,'acc_m',acc_m, 'lb[0,0]=',lb[0,0], 'Execution time: ',time.time()-start_time)
        acc_m = acc_acu/(n+1)
        container = container + [acc_m]
        print("indicator",max(container))
        print(' ')
        # print('******###################### w=',w,'j=',j,'prop=',prop,'Eps=',Eps,'***********','acc_m=',acc_m,'Execution time: ', time.time()-start_time)
        string_to_write = '****** w=' + str(w) + 'j=' + str(j) + 'prop=' + str(prop) + 'Eps=' + str(Eps) + "average accuracy=" + str(acc_m) + '**********' 'Execution time: ' + str(time.time()-start_time) + '\n'
        #results_file.write(string_to_write)

        # View the AutoML Leaderboard
        #print('lb.head',lb.head(rows=lb.nrows))  # Print all rows instead of default (10 rows)
        # Get the "All Models" Stacked Ensemble model
        #se = aml.leader
        # Get the Stacked Ensemble metalearner model
        #metalearner = h2o.get_model(se.metalearner()['name']) # the goal of the meta learner is to ignore what is not important.
results_file.close()

Parse progress: |█████████████████████████████████████████████████████████| 100%
num_testing_samples
&&&&&&&& w= 1 j= 2 &&&&&&&&&&&&&&&&&&&&&&&&&&&& Execution time:  7.028579473495483
N= 6 data.shape[0] 128 num_testing_samples= 19
test_ctl_pd.shape[0]= 128
e0 64
e1 24
e2 21


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 0.8553736209869385
test_ctl_pd.shape[0]= 109
e0 61
e1 25
e2 23
time.time()-start_time= 0.32030463218688965
test_ctl_pd.shape[0]= 90
e0 63
e1 22
e2 24
time.time()-start_time= 0.8766179084777832
test_ctl_pd.shape[0]= 71
e0 63
e1 22
e2 24
time.time()-start_time= 0.507498025894165
test_ctl_pd.shape[0]= 52
e0 63
e1 23
e2 23
time.time()-start_time= 0.2448422908782959
test_ctl_pd.shape[0]= 33
e0 59
e1 26
e2 24
time.time()-start_time= 0.5234570503234863
indicator 0.49122807017543857
 
N= 6 data.shape[0] 128 num_testing_samples= 19
test_ctl_pd.shape[0]= 128
e0 64
e1 24
e2 21


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 0.4723958969116211
test_ctl_pd.shape[0]= 109
e0 61
e1 25
e2 23
time.time()-start_time= 0.2630155086517334
test_ctl_pd.shape[0]= 90
e0 63
e1 22
e2 24
time.time()-start_time= 0.4331240653991699
test_ctl_pd.shape[0]= 71
e0 63
e1 22
e2 24
time.time()-start_time= 0.293886661529541
test_ctl_pd.shape[0]= 52
e0 63
e1 23
e2 23
time.time()-start_time= 0.21895051002502441
test_ctl_pd.shape[0]= 33
e0 59
e1 26
e2 24
time.time()-start_time= 0.2175753116607666
indicator 0.5087719298245614
 
N= 6 data.shape[0] 128 num_testing_samples= 19
test_ctl_pd.shape[0]= 128
e0 64
e1 24
e2 21


...

time.time()-start_time= 0.39583349227905273
test_ctl_pd.shape[0]= 109
e0 61
e1 25
e2 23
time.time()-start_time= 0.39638686180114746
test_ctl_pd.shape[0]= 90
e0 63
e1 22
e2 24
time.time()-start_time= 0.29735684394836426
test_ctl_pd.shape[0]= 71
e0 63
e1 22
e2 24
time.time()-start_time= 0.3714945316314697
test_ctl_pd.shape[0]= 52
e0 63
e1 23
e2 23
time.time()-start_time= 0.5101468563079834
test_ctl_pd.shape[0]= 33
e0 59
e1 26
e2 24
time.time()-start_time= 0.536318302154541
indicator 0.5701754385964911
 
N= 6 data.shape[0] 128 num_testing_samples= 19
test_ctl_pd.shape[0]= 128
e0 64
e1 24
e2 21


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 0.5015280246734619
test_ctl_pd.shape[0]= 109
e0 61
e1 25
e2 23
time.time()-start_time= 0.44640183448791504
test_ctl_pd.shape[0]= 90
e0 63
e1 22
e2 24
time.time()-start_time= 0.3061201572418213
test_ctl_pd.shape[0]= 71
e0 63
e1 22
e2 24
time.time()-start_time= 0.3358917236328125
test_ctl_pd.shape[0]= 52
e0 63
e1 23
e2 23
time.time()-start_time= 0.26468944549560547
test_ctl_pd.shape[0]= 33
e0 59
e1 26
e2 24
time.time()-start_time= 0.48241519927978516
indicator 0.5701754385964911
 
N= 6 data.shape[0] 128 num_testing_samples= 19
test_ctl_pd.shape[0]= 128
e0 64
e1 24
e2 21


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 0.6271231174468994
test_ctl_pd.shape[0]= 109
e0 61
e1 25
e2 23
time.time()-start_time= 0.2940027713775635


In [ ]:
while True:pass

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
var_imp=model.varimp(use_pandas=True).head(15)
var_imp.plot(kind='bar',x='variable',y='relative_importance')
plt.ylabel('Relative Importance')
plt.show

path = "/content/drive/My Drive/Computadora Virtual/Data Analytics/Investigacion/Parkinson/"
file= path + 'Results/6.9. Feature Relative Importance.svg'  # https://stackoverflow.com/questions/7608066/in-matplotlib-is-there-a-way-to-know-the-list-of-available-output-format
plt.savefig(file)

In [ ]:
model.partial_plot(train, cols=["SM579"], figsize=(5,5));